# Testing the Full, Last Layer and Subnetwork functionality

### Model training

In [2]:
import Pkg
Pkg.activate("../../../")

  Activating project at `c:\Users\marka\Documents\VSCode\Julia\LaplaceRedux.jl`

In [2]:
using Flux
using Flux: gradient
using LaplaceRedux
using LinearAlgebra
using Plots
using Statistics
using Zygote
using DelimitedFiles

ArgumentError: ArgumentError: Package Zygote not found in current path.
- Run `import Pkg; Pkg.add("Zygote")` to install the Zygote package.

In [3]:
xs, ys = LaplaceRedux.Data.toy_data_non_linear(200)
X = hcat(xs...) # bring into tabular format
data = zip(xs,ys)

zip([[3.3094690003501204, 1.105015603790398], [4.785206519692352, 2.302865692853331], [4.5978141179509695, 1.6641979932455095], [1.2039502150385224, 2.4143042079779162], [3.9535143318937274, 3.2749292394892926], [4.888228161913915, 4.798296383810684], [3.731562431596806, 4.8429509015736185], [0.7728393706989826, 2.5543798302040344], [4.734407081878071, 1.6562880772227127], [4.5508107698894085, 3.007820551587571]  …  [-4.001911977451673, 1.4413371112018547], [-1.401450781550467, 2.993436092989832], [-4.562711925033253, 2.145276692360886], [-0.662226465500714, 2.5663224474138024], [-1.1473346035389773, 2.490975960799372], [-3.9262243689688914, 4.612646203151002], [-3.2833710659982778, 4.90971289047463], [-3.141620549880873, 2.352464396233791], [-4.6898831470302955, 1.0777769882777402], [-2.5031164212937167, 1.389913242038404]], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [4]:
n_hidden = 10
D = size(X,1)
nn = Chain(
    Dense(D, n_hidden, σ),
    Dense(n_hidden, 1)
)
println(D)
print(Flux.params(nn))
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y) 

2


Params([

Float32[0.20560294 -0.69944406; 0.40646386 -0.33234173; -0.41245437 0.3885508; -0.29035503 0.23656149; -0.30834043 -0.1253471; 0.19812028 -0.14759278; -0.6857486 -0.08098635; -0.6883295 0.56313396; -0.4344271 -0.18774693; -0.4647752 -0.50013113], 

Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.16986336 -0.23157422 -0.67837626 0.43372405 0.0011231505 0.26485175 -0.39528698 -0.025584798 0.49851653 0.39710125], Float32[0.0]])

loss (generic function with 1 method)

In [5]:
using Flux.Optimise: update!, Adam
opt = Adam(1e-3)
epochs = 100
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
show_every = epochs/10

for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
  if epoch % show_every == 0
    println("Epoch " * string(epoch))
    @show avg_loss(data)
  end
end

Epoch 10

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(2 => 10, σ)
│   summary(x) = 2-element Vector{Float64}
└ @ Flux C:\Users\marka\.julia\packages\Flux\FWgS0\src\layers\stateless.jl:50



avg_loss(data) = 0.6622597560286522


Epoch 20
avg_loss(data) = 0.5813140459358692


Epoch 30
avg_loss(data) = 0.46266817778348923
Epoch 40
avg_loss(data) = 0.3480928037315607


Epoch 50
avg_loss(data) = 0.2602370107546449


Epoch 60
avg_loss(data) = 0.1983439323492348
Epoch 70
avg_loss(data) = 0.15517515078186989


Epoch 80
avg_loss(data) = 0.12445215816609562
Epoch 90
avg_loss(data) = 0.10208293938543647


Epoch 100
avg_loss(data) = 0.0855055350321345


### LA for Full Neural Network

In [6]:
la = Laplace(nn; likelihood=:classification, subset_of_weights=:all)
fit!(la, data)

la_untuned = deepcopy(la)   # saving for plotting
optimize_prior!(la; verbose=true, n_steps=500)

zoom=0
println("...")
p_plugin = plot(la, X, ys; title="Plugin", link_approx=:plugin, clim=(0,1))
p_untuned = plot(la_untuned, X, ys; title="LA - raw (λ=$(unique(diag(la_untuned.P₀))[1]))", clim=(0,1), zoom=zoom)
p_laplace = plot(la, X, ys; title="LA - tuned (λ=$(round(unique(diag(la.P₀))[1],digits=2)))", clim=(0,1), zoom=zoom)
plot(p_plugin, p_untuned, p_laplace, layout=(1,3), size=(1700,400))

### LA for Last Layer of Neural Network

In [ ]:
la = Laplace(nn; likelihood=:classification, subset_of_weights=:last_layer)
fit!(la, data)

la_untuned = deepcopy(la)   # saving for plotting
optimize_prior!(la; verbose=true, n_steps=500)

zoom=0
println("...")
p_plugin = plot(la, X, ys; title="Plugin", link_approx=:plugin, clim=(0,1))
p_untuned = plot(la_untuned, X, ys; title="LA - raw (λ=$(unique(diag(la_untuned.P₀))[1]))", clim=(0,1), zoom=zoom)
p_laplace = plot(la, X, ys; title="LA - tuned (λ=$(round(unique(diag(la.P₀))[1],digits=2)))", clim=(0,1), zoom=zoom)
plot(p_plugin, p_untuned, p_laplace, layout=(1,3), size=(1700,400))

### LA for Subset of Neural Network

Looking at the plot for tuned Laplace, by selecting just 4 of the 41 weights in the neural network provided in this notebook, we can see that there is not much information from the gradients and the jacobian that Laplace Approximation can use in order to adjust its predictive uncertainty.

In [ ]:
la = Laplace(nn; likelihood=:classification, subset_of_weights=:subnetwork, subnetwork_indices=[[3, 1, 1], [2, 8], [2, 9], [2, 10]])
fit!(la, data)

la_untuned = deepcopy(la)   # saving for plotting
optimize_prior!(la; verbose=true, n_steps=500)

zoom=0
println("...")
p_plugin = plot(la, X, ys; title="Plugin", link_approx=:plugin, clim=(0,1))
p_untuned = plot(la_untuned, X, ys; title="LA - raw (λ=$(unique(diag(la_untuned.P₀))[1]))", clim=(0,1), zoom=zoom)
p_laplace = plot(la, X, ys; title="LA - tuned (λ=$(round(unique(diag(la.P₀))[1],digits=2)))", clim=(0,1), zoom=zoom)
plot(p_plugin, p_untuned, p_laplace, layout=(1,3), size=(1700,400))

When providing all 41 weights, the uncertainties are much higher.

In [ ]:
la = Laplace(nn; likelihood=:classification, subset_of_weights=:subnetwork, subnetwork_indices=[[1, 2, 1], [1, 1, 1], [1, 1, 2], [1, 2, 2], [1, 3, 1], [1, 3, 2], [1, 4, 1], [1, 4, 2], [1, 5, 1], [1, 5, 2], [1, 6, 1], [1, 6, 2], [1, 7, 1], [1, 7, 2], [1, 8, 1], [1, 8, 2], [1, 9, 1], [1, 9, 2], [1, 10, 1], [1, 10, 2], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [3, 1, 1], [3, 1, 2], [3, 1, 3], [3, 1, 4], [3, 1, 5], [3, 1, 6], [3, 1, 7], [3, 1, 8], [3, 1, 9], [3, 1, 10], [4, 1]])
fit!(la, data)

la_untuned = deepcopy(la)   # saving for plotting
optimize_prior!(la; verbose=true, n_steps=500)

zoom=0
println("...")
p_plugin = plot(la, X, ys; title="Plugin", link_approx=:plugin, clim=(0,1))
p_untuned = plot(la_untuned, X, ys; title="LA - raw (λ=$(unique(diag(la_untuned.P₀))[1]))", clim=(0,1), zoom=zoom)
p_laplace = plot(la, X, ys; title="LA - tuned (λ=$(round(unique(diag(la.P₀))[1],digits=2)))", clim=(0,1), zoom=zoom)
plot(p_plugin, p_untuned, p_laplace, layout=(1,3), size=(1700,400))